In [1]:
#Run any one cell to select a certain url

In [ ]:
#If facing issues with the default firefox driver for webscraping using selenium , run the below code
!sudo apt-get install chromium-chromedriver
!sudo apt-get update
!sudo apt-get install chromium-chromedriver
#Here the chrome driver is being used , hence replace firefox with Chrome driver as given below
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=options)

In [1]:
#Alphabetical parse url
url = "https://www.espncricinfo.com/ci/content/player/country.html?country=6;alpha=A"

In [2]:
#Country based parse url
url = "https://www.espncricinfo.com/ci/content/player/index.html?country=6"

In [3]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd
import time
from selenium import webdriver

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
driver = webdriver.Firefox()

In [4]:
req = requests.get(url)

1.2.4
You should consider upgrading via the '/home/gokulvamsit/sem4-summer/IndicWiki/Gokul_IndicWiki/bin/python -m pip install --upgrade pip' command.


In [5]:
soup = bf(req.text,"html.parser")

In [6]:
search_tag_container = soup.find_all('td')

In [7]:
#Fetch total urls of players belonging to a country
u = list()
for job_elem in search_tag_container:
    u.append(job_elem.find('a'))
    #result.append(job_elem.text)
l = set(u)
l = list(l)

In [8]:
def evaluate(html):
  select_divs = html.find_all('div',class_="card overflow-hidden mb-3")
  exist = 0
  location = 0
  for i in range(len(select_divs)):
    check = ""
    try:
      check = select_divs[i].find('p',class_="benton-bold pl-3 pt-4 pb-3 m-0 player-card-header").text
    except:
      k = 1
    if(check == "Career Averages"):
      exist = 1
      location = i
      break
  return exist,select_divs[location]

In [9]:
# Handling images
def get_player_image(player_url):
    # For fixing installation errors on linux: `sudo apt install firefox-geckodriver`
    global driver
    driver.get(player_url)
    time.sleep(2)
    images = driver.find_elements_by_tag_name('img')
    images = [image for image in images if "player-card__face" in image.get_attribute("class")]
    print(images[0].get_attribute("src"))
    return ["image"], [images[0].get_attribute("src")]

In [10]:
# Get trophy info
def get_major_trophies_information(major_trophies_soup):
    major_trophies_results = {}
    all_h5_tags = major_trophies_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    required_h5_tag = all_h5_tags[0]
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "In Major Trophies":
            required_h5_tag = h5_tag
            break
    if str(required_h5_tag.text).strip() != "In Major Trophies":
        return ["Major trophies"], [[major_trophies_results]]    
    major_trophies_table_head = required_h5_tag.findNext('thead')
    major_trophies_table_headings = major_trophies_table_head.find_all('th')
    major_trophies_table_headings_count = 0
    major_trophies_headings_list = []
    for table_heading in major_trophies_table_headings:
        major_trophies_headings_list.append(table_heading.text)
    major_trophies_table_headings_count = len(major_trophies_headings_list)
    # print('Major Trophies table Head count', major_trophies_table_headings_count)
    current_title = "HOME"
    major_trophies_table_body = major_trophies_table_head.findNext('tbody')
    major_trophies_table_data = major_trophies_table_body.find_all('td')
    t = 0
    for table_data in major_trophies_table_data:
        attribute_number = t % major_trophies_table_headings_count
        if(attribute_number != 0):
            major_trophies_results[current_title][major_trophies_headings_list[attribute_number]] = table_data.text
        else:
            current_title = table_data.text
            major_trophies_results[current_title] = {}
        t += 1
    print(major_trophies_results)
    return ["Major trophies"], [[major_trophies_results]]

In [11]:
# Statistical analysis - Home, Away stats and function call to trophy info
def get_statistical_analysis_and_trophy_info(req):
    analysis_labels = []
    analysis_results = []
    analysis_url = req.url + "/bowling-batting-stats"
    #analysis_url = "https://www.espncricinfo.com/player/virat-kohli-253802/bowling-batting-stats"
    
    # selecting dropdown option
    global driver
    driver.get(analysis_url)
    # loading page
    time.sleep(2)
    dropdowns = driver.find_elements_by_class_name('dropdown-container')
    for dropdown in dropdowns:
        is_format_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Test", "ODI", "T20I", "Test+ODI+T20I"]
        is_role_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Batting", "Bowling", "Fielding", "Allround"]
        if (not is_format_dropdown) and (not is_role_dropdown):
            continue
        required_span_text = "Test+ODI+T20I"
        if is_role_dropdown:
            required_span_text = "Allround"
        dropdown.find_elements_by_tag_name('button')[0].click()
        all_list_items = dropdown.find_elements_by_tag_name('li')
        for list_item in all_list_items:
            span_text = list_item.find_elements_by_tag_name('span')[0].text
            if span_text == required_span_text:
                list_item.click()
                break
    # waiting for changes to load
    time.sleep(2)
    analysis_page = driver.page_source
    analysis_soup = bf(analysis_page, "html.parser")
    
    all_h5_tags = analysis_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    required_h5_tag = all_h5_tags[0]
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "Home vs Away":
            required_h5_tag = h5_tag
            break
    if str(required_h5_tag.text).strip() != "Home vs Away":
        return get_major_trophies_information(analysis_soup)
    analysis_table_head = required_h5_tag.findNext('thead')
    analysis_table_headings = analysis_table_head.find_all('th')
    analysis_table_headings_count = 0
    analysis_table_headings_list = []
    for table_heading in analysis_table_headings:
        analysis_table_headings_list.append(table_heading.text)
    analysis_table_headings_count = len(analysis_table_headings_list)
    # print('Analysis table Head count', analysis_table_headings_count)
    current_title = "HOME"
    analysis_table_body = analysis_table_head.findNext('tbody')
    analysis_table_data = analysis_table_body.find_all('td')
    t = 0
    for table_data in analysis_table_data:
        attribute_number = t % analysis_table_headings_count
        if(attribute_number != 0):
            analysis_results.append(table_data.text)
            analysis_labels.append(current_title + "_" + analysis_table_headings_list[attribute_number])
        else:
            current_title = table_data.text.upper()
        t += 1
    # for i in range(len(analysis_labels)):
    #     print(f'{analysis_labels[i]} : {analysis_results[i]}')
    trophy_label, trophy_result = get_major_trophies_information(analysis_soup)
    return analysis_labels + trophy_label, analysis_results + trophy_result

In [12]:
def find_relations(Relations):
    relations_container = Relations.find_next('div')
    individual_relations = relations_container.find_all('span')
    total_relations = list()
    for i in range(len(individual_relations)):
        name = individual_relations[i].find('h5').text
        bond = individual_relations[i].find('p').text
        total_relations.append([name,bond])
    return total_relations

In [13]:
#Get relevant information related to a player using each player's url
def load(req,final_list_players,ind,u,debut_url):
    lab = list()
    result = list()
    soup = bf(req.text,"html.parser")
    print(soup.title)
    search_tag_container = soup.find_all('h5',class_='player-card-description gray-900')
    count = 0
    for job_elem in search_tag_container:
        result.append(job_elem.text)
        count+=1
    search_tag_container = soup.find_all('p',class_="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading")
    for job_elem in search_tag_container:
        lab.append(job_elem.text)
    ##################################
    Relations = soup.find('p',class_="text-uppercase gray-700 mb-0 player-card-heading text-uppercase")
    #relation_players = soup.find('p',class_="player-card-description player-description-link gray-900 m-0 player-link")
    if(Relations != None):
        if(Relations.text.lower()=='relations' or Relations.text.lower()=='relation'):
            lab.append("Relations")
            all_relations = find_relations(Relations)
            result.append([all_relations])
    ######################################
    rec = extra(u)
    lab.append("Records")
    result.append([rec])
    print(result)
    ##########################################
    records_format , records_data = format_records(u)
    for iter in range(len(records_format)):
      lab.append(records_format[iter])
      result.append([records_data[iter]])
    ###########################################
    matches = debut(debut_url)
    result.append([matches])
    lab.append("Matches")
    ##########################################
    lab.append("Teams")
    team_data = teams_data(soup)
    result.append([team_data])
    ###########################################
    analysis_and_trophy_labels, analysis_and_trophy_results = get_statistical_analysis_and_trophy_info(req)
    lab = lab + analysis_and_trophy_labels
    result = result + analysis_and_trophy_results
    ###########################################
    player_image_label, player_image_result = get_player_image(req.url)
    lab = lab + player_image_label
    result = result + player_image_result
    ###########################################
    career_span_tag = soup.find('div', class_="intl_career-desktop")
    career_span_tag_text = career_span_tag.findNext('p').text
    career_span_info = ""
    start_index = 0
    for i in range(len(career_span_tag_text)):
        if career_span_tag_text[i] == '1' or career_span_tag_text[i] == '2':
            start_index = i
            break
    career_span_tag_text = career_span_tag_text[start_index:]
    result.append(career_span_tag_text)
    lab.append("career_span")
    ###########################################
    search_tag_container = soup.find('div',class_="player-card__details")
    l = list()
    for job_elem in search_tag_container:
        l.append(job_elem.text)
        count+=1
    p = l[1].split("|")
    l.pop()
    l.pop()
    l = l+p
    result = result + l
    l_label = ["Player Name" , "Nationality ", "player-type"]
    lab = lab + l_label
    ##########################
    valid,soup = evaluate(soup)
    if(valid == 1):
        search_tag_container = soup.findAll('thead')
        for m in range(len(search_tag_container)):
            actual_heading = search_tag_container[m].findPrevious('h5').text
            current_string = "_"
            if "Bowling" in actual_heading:
                current_string = "Bowling_"
            elif "Batting" in actual_heading:
                current_string = "Batting_"
            else:
                continue
            tag_content = search_tag_container[m].find_all('th')
            head = 0
            table_labels = list()
            table_labels_1 =list()
            for job_elem in tag_content:
                table_labels.append(job_elem.text)
                head+=1
            print('Head count',head)

            tag_content = search_tag_container[m].findNext('tbody').find_all('td')
            track_head_label = 0
            current_format = ""
            for job_elem in tag_content:
                attribute_number = track_head_label % head
                if(attribute_number != 0):
                    result.append(job_elem.text)
                    lab.append(current_string + current_format + '_' + table_labels[attribute_number])
                    count += 1
                else:
                    current_format = job_elem.text
                track_head_label += 1
    ####################################
      
    print('label--',len(lab))
    print('result--',len(result))
    
    lo = dict(zip(lab,result))
    lp = pd.DataFrame(lo,index=[ind])
    final_list_players.append(lp)
    print('count',count)

In [14]:
#Get individual player's "Records" information.
import re
def extra(l):
    re = requests.get(l)
    time.sleep(2)
    so = bf(re.text,"html.parser")
    res1 = so.find_all('div',class_='col-14 d-flex flex-row')
    res = so.find_all('div',class_='row bg-gray-100 class-record-row-p d-flex align-items-center')
    #print(res1)
    url_list = list()
    for i in range(len(res1)):

        pre = res1[i].find_all('h1')
        loi = res1[i].find_all('p',class_='record-class-title m-0')
        #print(loi)    
        for j in range(len(pre)):
            #print(pre[j].text)
            #print(loi[j].text)
            url_list.append(str(pre[j].text)  +str(" ") + str(loi[j].text))
    return url_list


In [15]:
def format_records(url):
  req = requests.get(url)
  soup = bf(req.text,"html.parser")
  record_sections = soup.find_all('div',class_="card content-block")
  required = ['Test Records','ODI Records','T20I Records']
  available_labels = list()
  labels_data = list()
  count = -1
  for i in range(len(record_sections)):
    name = record_sections[i].find('h5',class_="m-0")
    if(name.text in required):
      available_labels.append(name.text)
      labels_data.append([])
      count +=1
      individual_block_record = record_sections[i].find_all('a',class_="gray-900 benton-normal d-block")
      for i in range(len(individual_block_record)):
        try:
          a = individual_block_record[i].find('span',class_="m-0 benton-bold gray-1000 pr-3").text
          b = individual_block_record[i].find('span',class_="list-record-title").text
          c = individual_block_record[i].find('span',class_="m-0 ml-1 gray-600").text
          labels_data[count].append(str(a)+str(" ")+str(b)+str(" ")+str(c))
        except:
          no = 1
  return available_labels , labels_data

In [16]:
def teams_data(so):
  teams = []
  loi = so.find_all('h5',class_='m-0 ml-2 link-border-bottom player-description-link')
  for i in range(len(loi)):
    teams.append(loi[i].text)
  return teams

In [17]:
def debut(link):
  player_match = list()
  url = link
  request = requests.get(url)
  soup = bf(request.text,"html.parser")
  search_tag_container = soup.find_all('div',{'id':'debut-last-matches'})
  cp_jo = search_tag_container
  if(len(search_tag_container) != 0):
    total = cp_jo[0].find_all('div',class_="")
    for i in range(len(total)):
      p =total[i]
      a= p.find('h5',{'class':"benton-bold player-matches-subtitle d-flex justify-content-between align-items-center player_matches-dropdown m-0"})
      k = total[i].find_all('span',class_="match-event-row-debut-last white-space-nowrap")
      search_tag_container = total[i].find_all('span',{"class": "player-match-link"})
      new = total[i].find_all('div',{"class":"col-12 player-match-event-cell white-space-nowrap"})
      #print(a)
      #print(k[0])
      #print(new)
      try:
        l =a.span.decompose()
      except:
        player_match.append(a.text)
        #print(a.text)
      else:
        player_match.append(a.text)
        #print(a.text)
      for j in range(len(k)):
        try:
          temp = search_tag_container[j].text
        except:
          temp = new[j].text
        append = k[j].text + str(" ") + temp
        player_match.append(append)
        #print(k[j].text,end=" ")
        #print(search_tag_container[j].text)
  return player_match

In [ ]:
import time
final_list_players = list()
for i in range(10):
    u = "https://www.espncricinfo.com"
    rl = l[i]["href"]
    link = u+rl
    print(link)
    #link = "https://www.espncricinfo.com/player/virat-kohli-253802"
    req = requests.get(link)
    time.sleep(2)
    print('player-',i,end=" ")
    ind = i
    record_url = req.url+str("/tests-odi-t20-records")
    debut_url = req.url+str("/matches")
    #debut(debut_url)
    load(req,final_list_players,ind,record_url,debut_url)
    #print(record_url)
    #jk = extra(record_url)
    #print(jk)
driver.close()

https://www.espncricinfo.com/ci/content/player/625371.html
player- 0 <title>Hardik Pandya profile and biography, stats, records, averages, photos and videos</title>
['Hardik Himanshu Pandya', 'October 11, 1993, Choryasi, Gujarat', '27y 230d', 'Right hand bat', 'Right arm medium fast', 'Allrounder', [[['KH Pandya', ' (brother)']]], [['4th Highest career strike rate (117.31)']]]
{'World Cup': {'Span': '2019-2019', 'Mat': '9', 'Runs': '226', 'HS': '48', 'Bat Avg': '32.28', '100s': '0', 'Wkts': '10', 'BBI': '3/60', 'Bowl Avg': '44.70', '5w': '0', 'Ct': '3', 'St': '0', 'Avg Diff': '-12.41'}, 'Asia Cup': {'Span': '2016-2018', 'Mat': '6', 'Runs': '33', 'HS': '31', 'Bat Avg': '11.00', '100s': '0', 'Wkts': '7', 'BBI': '3/8', 'Bowl Avg': '18.14', '5w': '0', 'Ct': '3', 'St': '0', 'Avg Diff': '-7.14'}, 'ICC Champions Trophy': {'Span': '2017-2017', 'Mat': '5', 'Runs': '105', 'HS': '76', 'Bat Avg': '52.50', '100s': '0', 'Wkts': '4', 'BBI': '2/43', 'Bowl Avg': '58.25', '5w': '0', 'Ct': '1', 'St': '0'

player- 9 <title>Yuzvendra Chahal profile and biography, stats, records, averages, photos and videos</title>
['Yuzvendra Singh Chahal', 'July 23, 1990, Jind, Haryana', '30y 310d', 'Right hand bat', 'Legbreak googly', 'Bowler', [['8th Worst economy rate in an innings (12.36)', '4th Outstanding bowling analyses in an innings (6/25)', '1st Most consecutive four-wickets-in-an-innings (2)', '5th Most runs conceded in an innings (64)', '6th Most wickets taken stumped (9)', '5th Fastest to 50 wickets (34)']]]
{'World Cup': {'Span': '2019-2019', 'Mat': '8', 'Runs': '5', 'HS': '5', 'Bat Avg': '5.00', '100s': '0', 'Wkts': '12', 'BBI': '4/51', 'Bowl Avg': '36.83', '5w': '0', 'Ct': '4', 'St': '0', 'Avg Diff': '-31.83'}, 'Asia Cup': {'Span': '2018-2018', 'Mat': '5', 'Runs': '-', 'HS': '-', 'Bat Avg': '-', '100s': '-', 'Wkts': '6', 'BBI': '3/46', 'Bowl Avg': '32.83', '5w': '0', 'Ct': '2', 'St': '0', 'Avg Diff': '-'}}
https://img1.hscicdn.com/image/upload/f_auto,t_gn_f_345/lsci/db/PICTURES/CMS/319900

In [ ]:
a = final_list_players[0]

In [ ]:
a

In [ ]:
df = a.T
for i in range(1,len(final_list_players)):
    lo = final_list_players[i].T
    df = pd.concat([df,lo],axis=1)

In [ ]:
df.T.shape
df.T.columns.tolist()

In [ ]:
df.T